Load train data and test data by pandas

In [1]:
import pandas as pd 

trainData = pd.read_csv('Data/train.csv')
#trainData.head()
testData = pd.read_csv('Data/test.csv')
#testData.head()

Get the label from train data and remove the column

In [2]:
label = trainData['label'].astype('long')
trainData = trainData.drop('label',axis=1)

Convert dataframe in pytorch tensor

In [3]:
import numpy as np
import torch
import torch.utils.data


train = torch.utils.data.TensorDataset(torch.Tensor(np.array(trainData)), torch.Tensor(np.array(label)))
train_loader = torch.utils.data.DataLoader(train, batch_size = 64, shuffle = True)

Define the network

In [7]:
from torch import nn, optim
import torch.nn.functional as F

class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        # make sure input tensor is flattened
        #x = x.view(x.shape[0], -1)
        
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

Train the network

In [8]:
model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

epochs = 15
steps = 0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_losses = []
for e in range(epochs):
    running_loss = 0
    for images, labels in train_loader:
        
        optimizer.zero_grad()
        labels = labels.to(device=device, dtype=torch.int64)

        log_ps = model(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    else:

        train_losses.append(running_loss/len(train_loader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(train_losses[-1]))

Epoch: 1/15..Training Loss: 0.628..
Epoch: 2/15..Training Loss: 0.366..
Epoch: 3/15..Training Loss: 0.363..
Epoch: 4/15..Training Loss: 0.334..
Epoch: 5/15..Training Loss: 0.327..
Epoch: 6/15..Training Loss: 0.305..
Epoch: 7/15..Training Loss: 0.322..
Epoch: 8/15..Training Loss: 0.333..
Epoch: 9/15..Training Loss: 0.318..
Epoch: 10/15..Training Loss: 0.336..
Epoch: 11/15..Training Loss: 0.303..
Epoch: 12/15..Training Loss: 0.317..
Epoch: 13/15..Training Loss: 0.294..
Epoch: 14/15..Training Loss: 0.291..
Epoch: 15/15..Training Loss: 0.289..


Convert test data for prediction and dump into submission

In [9]:
test = torch.Tensor(np.array(testData))
test.shape
preds = []

with torch.no_grad():
    model.eval()
    for val in test:
        val = val.view(1, 784)
        y_hat = model.forward(val)
        preds.append(y_hat.argmax().item())

In [12]:
results = pd.Series(preds,name="Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("submission.csv",index=False)